In [1]:
%%capture
!pip install ultralytics
import ultralytics
ultralytics.checks()

import os
import glob


In [ ]:
# SETTING VARIABLES

# dimensions of frames
img_width, img_height = 1920, 1080
# video sequence name - "simple_mid", "simple_ben", "difficult_mid", "difficult_ben"
vid_seq_name = "difficult_mid"

In [ ]:
# DEFINING FUNCTION
import os
import glob

def yolo_to_mot(yolo_labels, output_mot):
    # unnormalize labels
    count = 0
    frame_files = sorted(glob.glob(os.path.join(yolo_labels, "*.txt")))
    with open(output_mot, "w") as out_file:
        for file in frame_files:
            name = os.path.basename(file).split(".")[0]
            frame_idx = int(name.split("_")[-1])
            with open(file, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 6:
                        print(f"Skipping line in {file}: {line.strip()}")
                        count = count + 1
                        continue
                    class_id, x_center, y_center, width, height, track_id = map(float, line.strip().split())
                    #calculate
                    x = (x_center - width / 2) * img_width
                    y = (y_center - height / 2) * img_height
                    w = width * img_width
                    h = height * img_height

                    out_file.write(f"{frame_idx}, {int(track_id)}, {x:.2f}, {y:.2f}, {w:.2f}, {h:.2f}, 1, 1, -1\n")
    print("number of skipped lines: ", count)

#set video path
video_paths = {
    "simple_mid": "../videos/simple_mid/Midwater_simple_V4455_20221130T192123Z_prores.mov",
    "simple_ben": "../videos/simple_ben/10s_Benthic_2_V4289_20200729T185027Z.mov",
    "difficult_mid": "../videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov",
    "difficult_ben": "../videos/difficult_ben/Benthic_1_V4277_20200219T211238Z.mov"
}
vid_path = video_paths.get(vid_seq_name)
if vid_path is None: raise ValueError(f"invalid name: {vid_seq_name}")

In [ ]:
# MODEL + TRACKER
from ultralytics import YOLO

# Load a model
model = YOLO('/Users/eliu/Desktop/mbari_452k_yolov8.pt')
results = model.track(source=vid_path,
                # tracker="/Users/eliu/Desktop/bytetrack_kwalz_settings.yaml",
                tracker="bytetrack.yaml", 
                line_width=1,
                agnostic_nms=True,
                #save_crop=True,
                #save_conf=True,
                save_txt=True,
                save=True,
                #save_frames=True,
                conf=0.025,
                iou=0.4,
                half=True,
                device='mps',
                #vid_stride=60,
                #show=True,
                #visualize=True,
                #stream=True,
                imgsz=1280
                )
labels_dir = results[0].save_dir 

#paths
yolo_labels = labels_dir + '/labels' # where the saved labels are
output_mot = f"./data/trackers/mot_challenge/MOT17-train/YOLOTracker/{vid_seq_name}.txt" #where output motchallenge file will go

yolo_to_mot(yolo_labels, output_mot)

number of skipped lines:  0


In [ ]:
# GROUND TRUTH - adding track id to exported gt yolo txt files

#gt paths
input_folder = "../temp/labels"
output_folder = "../temp/labels_w_trackid"
os.makedirs(output_folder, exist_ok=True)

#get all name files
name_files = [f for f in os.listdir(input_folder) if f.endswith("_name.txt")]
for name_file in name_files:
    #paths
    base_file = name_file.replace("_name", "")
    name_file_path = os.path.join(input_folder, name_file)
    base_file_path = os.path.join(input_folder, base_file)
    output_file_path = os.path.join(output_folder, base_file)

    # reading files
    with open(name_file_path, "r") as f_name, open(base_file_path, "r") as f_base:
        name_lines = f_name.readlines()
        base_lines = f_base.readlines()

    #make new files
    with open(output_file_path, "w") as f_out:
        for base_line, name_line in zip(base_lines, name_lines):
            if base_line.strip() and name_line.strip():
                # get track id
                track_id = int(name_line.strip().rsplit("-", 1)[-1])
                # combine line + track id
                f_out.write(base_line.strip() + f" {track_id}\n")

# gt paths
yolo_labels = output_folder # where the saved labels are
output_mot = f"./data/gt/mot_challenge/MOT17-train/{vid_seq_name}/gt/gt.txt" # where the motchallenge gt file will go

#Converting format
yolo_to_mot(yolo_labels, output_mot)

number of skipped lines:  0
